# Python Type Annotatios

 * Python now supports annotation the expected types.

# What you will see
* How it looks
* What do you need for it
* Real word experience
* How can you start using it

# Life without annotations.

In [1]:
def foo(a, b):
    a += b
    return a

In [2]:
lines = ["This time"]
var = "correctly with comma",

foo(lines, var)

compare to

In [3]:
lines = ["aaa"]
var = "incorrect"

foo(lines, var)

['aaa', 'i', 'n', 'c', 'o', 'r', 'r', 'e', 'c', 't']

## The joy of life with annotations

```python
from typing import List

def foo(a: List[str], b: List[str]) -> List[str]:
    a += b
    return a

foo([], "")
```
---    
```python
MyPy: error: Argument 2 to "foo" has incompatible type "str"; 
        expected "List[str]"
```


 * Note the code has not been executed. MyPy is static checker.

* It can **today** show you a bug that otherwise could appear
     * sporadically
     * under certain circumstances

 * Helps the editor with hinting

* Helps with doing refactoring in big projects.

# Some of the Checkers available
 * MyPy
  * can check only annotated code
 * pytype
  * Can infer the types
  * Not good structural typing (Duck typing)
 * PyCharm
  * there is a commandline only interface for it. Hidden a bit.
  ```
  $ pycharm.sh inspect . ~/.idea/inspectionProfiles/profiles_settings.xml results -format plain
  ```
  
 * pyre
 * ...

# Pycharm
  * type infering -- for the hinting purposes it is really quite good


Hints for `Any` type
![missing locator](pics/pycharm_any_hints.png "No hints when using `Any`")

Hints for List type
![missing locator](pics/pycharm_list_hints.png "No hints when using `Any`")

## A real-world bug found.
![missing locator](pics/missing_locator.png "Missing locator")

  * Not without problems though. I found:
   * problem with inferring the retval type of `functools.partial` from the passed `Callable`
   * *Descriptors* - objects with the `__get__` don't get the correct type when accessed as attribute. [BUG](https://youtrack.jetbrains.com/issue/PY-26184)

## The problem with the Descriptor object not getting correct type

![pycharm_descriptors](pics/pycharm_descriptor_not_working.png "Pycharm and descriptors")

A workaround for this.
![pycharm_descriptors_workaround](pics/pycharm_descriptor_sloppy_workaround.png "Pycharm and descriptors workarond")

# The `typing` library

 * there are the buildins: `str`, `int`, `float`, `bytes`
 * Ther are many things already defined in `typing`:
   * `Any` is the most permissive but ambiguous type.
   * `Sequence`, `List`, ...
   * `Dict`, `Mapping`, `MappingView`, ...
   * `Union`, `Optional`, ...
   * `Callable`, `Generator`, `AsyncGenerator`, ...
   * `Type`
   * ...

# Some examples of how to annotate.

## The Optional

In [4]:
from typing import Optional

class Duck: pass


# Identical to
#   deref_none(duck: Union[None, Duck]):

def deref_none(duck: Optional[Duck]):
    duck.talk()

    
try:
    deref_none(None)
except AttributeError as exc:
    print(exc)

'NoneType' object has no attribute 'talk'


```
$ mypy ducks.py
ducks.py:48: error: Item "None" of "Optional[Duck]" has no attribute "talk"
Found 1 error in 1 file (checked 1 source file)
```

```
$ pytype --protocols ducks.py
...
File "/home/jhenner/work/types_talk/ducks.py", line 48, in deref_none: No attribute 'talk' on None [attribute-error]
  In Optional[Duck]

For more details, see https://google.github.io/pytype/errors.html#attribute-error.
ninja: build stopped: subcommand failed.
```

In [5]:
def deref_none(duck: Optional[Duck]):
    if duck is None:
        return
    duck.talk()

It is fixed now.
 * Checker won't complain.
 * Code will not deref the `None` dereferencing here.

 * How many times did I see 
    * ```AttributeError: 'NoneType' object has no attribute [...]``` 
    * ```NoMethodError (undefined method [...] for nil:NilClass)```
    * There is over 1000 hits on Bugzilla for each of them
 * I think static checkers pretty much sovles the curse of `null` / `Nil` / `None`.
        

## The `TypeVar`

In [6]:
from typing import Sequence, TypeVar

# Declare type variable
T = TypeVar('T', int, float, str)

def double(a: T) -> T:
    return a * 2

this means that what your pass in is what you get out

In [7]:
T = TypeVar('T')
def first(l: Sequence[T]) -> T:   # Generic function
    return l[0] 

## Some kind of container for thing of a kind.
*Pun intended*

In [8]:
from typing import TypeVar, Generic

T = TypeVar('T')

class Box(Generic[T]):
    def __init__(self, value: T) -> None:
        self.value = value

    def get(self) -> T:
        return self.value

box = Box(3)  # The type of box is now Box[int]

box.get()     # The type of result is int. The value of T is bound by the call to the Box class

3

## Defining a variable to hold a callable

In [9]:
from typing import Callable

a_call: Callable[[int, str], float]

def the_func(arg: int, str) -> float:
    return 1. * arg
    
a_call = the_func 
f:float = a_call(1, "")

A factory.
 * Note the `cls: Type[T]` means `cls` will contain **the type** `T` (some class)
 * Contrary: `some_object: T` would denote the `some_object` **is of type `T`**

In [10]:
from typing import Generic, T, Type, List

class Factory(Generic[T]):
    def __init__(self, cls: Type[T], *args, **kwargs):
        self.cls = cls
        self.args = args
        self.kwargs = kwargs
        
    def __call__(self) -> T:
        return self.cls(*self.args, **self.kwargs)
    
i: List[int] = Factory(list, (1,))()

## Duck typing

 * If it walks like a duck and it quacks like a duck, then it must be a duck.
 * there are Protocols that allow the *duck typing*

* We will make really unrealated classes: `Duck` and `Lure`... and a `BrokenDuck`.
* Let's define a `Qack` and somewhat related `LureQuack`

In [11]:
from typing import Protocol
from abc import abstractmethod, ABCMeta


class Quack:
    def __init__(self, sound_level: int):
        self.sound_level = sound_level


class LureQuack(Quack):
    pass


Now let's define the Duck and Lure and Something that isn't working as a Duck for us.

In [12]:
# Note: no explicit inheritance from the CanQuack protocol.

class Duck:
    sound_level = 1

    def talk(self) -> Quack:
        return Quack(self.sound_level)


class Lure:
    sound_level = 100

    def talk(self) -> LureQuack:
        return LureQuack(self.sound_level)

* We want to be able to use the same callable for the `Duck` and `Lure`.

In [13]:
# ABCMeta needed for PyType https://github.com/google/pytype/issues/214
class CanQuack(Protocol):
    sound_level: int

    def talk(self) -> Quack: ...
        

def use_as_duck(duck: CanQuack):
    return duck.talk()

In [14]:
class BrokenDuck:
    sound_level = 0

    def talk(self) -> object:  # Note: Doesn't return Quack
        return object()


def use_as_duck(duck: CanQuack):
    return duck.talk()

use_as_duck(Duck())        # Passes typecheck
use_as_duck(Lure())        # Passes typecheck
use_as_duck(BrokenDuck())  # This should fail on typechecking. No problem in runtime though.

```
$ mypy ducks.py
ducks.py:49: error: Argument 1 to "use_as_duck" has incompatible type "BrokenDuck"; expected "CanQuack"
ducks.py:49: note: Following member(s) of "BrokenDuck" have conflicts:
ducks.py:49: note:     Expected:
ducks.py:49: note:         def talk(self) -> Quack
ducks.py:49: note:     Got:
ducks.py:49: note:         def talk(self) -> object
Found 1 error in 1 file (checked 1 source file)
```

```
$ pytype --protocols ducks.py
Computing dependencies
Analyzing 1 sources with 0 local dependencies
ninja: Entering directory `/home/jhenner/work/types_talk/.pytype'
[1/1] check ducks
Success: no errors found
```

Why not to use the *nominal inheritance* (Interfaces)?
 * You can still subclass a protocol. You dont't need to though...
 * The `Duck` can be defined in library that you don't own. The `Duck` doesn't really need to have a clue about existance of that protocol in it's definition time.

# Gradual addition of type annotations


 * Allows existing projects to adopt typing

 * The `*.pyi` stubfiles
  * like `*.h` files known from C.
  * Just defining the "interfaces". No real code.
  * Can be generated or (perhaps also with infering) using **pytype** or **stubgen**
  * Can be merged into the ".py" files.

 * The marker file `py.typed` is distributed in python packages to note the package is annotated.
  * MyPy will not load the types from it otherwise

## Gradual typing annotations with Mypy

precommit selecting specific files:

*.pre-commit-config.yaml*

```yaml
- repo: https://github.com/pre-commit/mirrors-mypy
  rev: v0.782
  hooks:
  - id: mypy
    files: cfme/utils/appliance
```

*.mypy.ini*
```ini
[mypy]
ignore_missing_imports = True
follow_imports = skip
```


# The Takeaways
 * Well the developer **need to learn yet another thing**. It does take a time and effort.
 * It allows put the **thinking about the types** *on the paper* and have the code checked.
 * It certainly **does prevent some bugs**
 * Just add a check of some basic parts of your projects and have them ran for each PR request.
 

# References
 * [PEP 483 - The theory of type hints](https://www.python.org/dev/peps/pep-0483)
 * [PEP 484 - The type hints](https://www.python.org/dev/peps/pep-0484/)
 * [Python typing library docs](https://docs.python.org/3/library/typing.html)



# A bonus: Annotations usage in runtime.

In [15]:
import inspect
from typing import Dict, List, Union

def foo(a: str, b:Dict[str, Union[float, int]]={}) -> int: pass

sig = inspect.signature(foo)
sig.parameters, sig.return_annotation

(mappingproxy({'a': <Parameter "a: str">,
               'b': <Parameter "b: Dict[str, Union[float, int]] = {}">}),
 int)

In [16]:
try:
    from typing import get_args
except ImportError:
    print("Old Python")
    def get_args(t):
        return t.__args__


get_args(sig.parameters['b'].annotation)

(str, typing.Union[float, int])

In [17]:
from typing import Dict, TypeVar, T, Optional

def foo(a: Optional[T]) -> T: pass
a_annots = inspect.signature(foo).parameters['a'].annotation
a_annots.__args__, a_annots.__parameters__

((~T, NoneType), (~T,))

# Type-Based Fulfilling of test fixtures

In [18]:
from typing import Callable
from dataclasses import dataclass
from random import choice

@dataclass
class RandomWord:
    def __init__(self):
        self.val = choice("Lorem ipsum dolor sit amet consectetur adipiscing elit".split())
    def __repr__(self):
        return self.val
 
@dataclass
class Surname:
    value: RandomWord

@dataclass
class Fixture:
    name: RandomWord
        
@dataclass
class ExtendedFixture(Fixture):
    surname: Surname
    pass

In [19]:
def satisfy_the_annotations(f: Callable):
    sig = inspect.signature(f)
    params = sig.bind(**{n: satisfy_the_annotations(p.annotation)
                         for n, p in sig.parameters.items()})
    return f(*params.args, **params.kwargs)


def run_test(f: Callable):
    retval = satisfy_the_annotations(f)
    return "PASSED"
        
    
def test_someting_very_nice(somefixture: Fixture,
                            extended: ExtendedFixture,
                            extended2: ExtendedFixture):
    print(somefixture)
    print(extended)
    print(extended2)
    assert isinstance(somefixture.name, RandomWord)

    
run_test(test_someting_very_nice)

Fixture(name=sit)
ExtendedFixture(name=Lorem, surname=Surname(value=adipiscing))
ExtendedFixture(name=adipiscing, surname=Surname(value=elit))


'PASSED'

I made an [issue](https://github.com/pytest-dev/pytest/issues/7690#issuecomment-681040282) suggesting this in pytest project

I have got the issue closed, but they do seem to want [something like that](https://fastapi.tiangolo.com/tutorial/dependencies/)